In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd drive/'My Drive'/personal_projects/ta-lib

/content/drive/My Drive/personal_projects/ta-lib


In [ ]:
!dpkg -i libta.deb ta.deb
!pip install ta-lib

Selecting previously unselected package libta-lib0.
(Reading database ... 144676 files and directories currently installed.)
Preparing to unpack libta.deb ...
Unpacking libta-lib0 (0.4.0-oneiric1) ...
Selecting previously unselected package ta-lib0-dev.
Preparing to unpack ta.deb ...
Unpacking ta-lib0-dev (0.4.0-oneiric1) ...
Setting up libta-lib0 (0.4.0-oneiric1) ...
Setting up ta-lib0-dev (0.4.0-oneiric1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Processing triggers for libc-bin (2.27-3ubuntu1.2) ...
/sbin/ldconfig.real: /usr/local/lib/python3.6/dist-packages/ideep4py/lib/libmkldnn.so.0 is not a symbolic link

     |████████████████████████████████| 276kB 2.8MB/s 
  Created wheel for ta-lib: filename=TA_Lib-0.4.19-cp36-cp36m-linux_x86_64.whl size=1437787 sha256=43a340456cedee6559b16c9e2f8811fadbee6ce6c0f636cd31ae97b7fab26b9a
  Stored in directory: /root/.cache/pip/wheels/a3/f6/12/3d1ccd06caadd8fa47e016991dd0d27f1163bb260f1854e2ff
Successfully built ta-lib


In [ ]:
%cd ..

/content/drive/My Drive/personal_projects


In [ ]:
!nvidia-smi

Tue Sep 29 12:21:51 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.66       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8    26W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# common imports

import numpy as np
import os
import pickle
import random
import numpy as np
import math
from talib import NATR, RSI

In [ ]:
import tensorflow as tf
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Masking, Concatenate, Conv2D, MaxPooling2D, Flatten
from keras.layers.core import Activation, Dropout, Reshape
from keras.layers.advanced_activations import LeakyReLU
from keras.callbacks import EarlyStopping, ModelCheckpoint
# from hyperas import optim
# from hyperas.distributions import choice, uniform
# from hyperopt import Trials, STATUS_OK, tpe
from keras import optimizers
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

In [62]:
# model_interface.py

# This is an abstract class. You need to implement yours.
class AbstractModelBuilder(object):

  def __init__(self, weights_path = None):
    self.weights_path = weights_path
    self.model = None

  def loadModel(self):
    weights_path = self.weights_path
    self.model = self.buildModel()

    if weights_path and os.path.isfile(weights_path):
      try:
        self.model.load_weights(weights_path)
      except e:
        print(e)

  # You need to override this method.
  def buildModel(self):
    raise NotImplementedError("You need to implement your own model.")

  def predict(self, data):
    raise NotImplementedError("You need to implement predictor function")

  def compile_model(self):
    raise NotImplementedError("You need to implement this function")

  def trainModel(self, inputs, targets):
    raise NotImplementedError("You need to implement this function")
  
  def printSummary(self):
    raise NotImplementedError("You need to implement this function")

In [60]:

# do a cut off
# unable to build (PnL transfer t to t+1)?
# Doesn't work
# class lstm_model(AbstractModelBuilder):
  
#   def buildModel(self):
#       inputs = []
    
#       weekly_inputs = Input(shape=(None, num_encoder_tokens))
#       inputs.append(weekly_inputs)
    
#       encoder_inputs = Dense(8, activation='relu')(weekly_inputs)
#       week_encoder = LSTM(16, return_state=True)
#       encoder_outputs, state_h, state_c = week_encoder(encoder_inputs)
#       encoder_states = [state_h, state_c]

#       input_price_info = Input(shape=(None, 3))
#       inputs.append(input_price_info)
#       inputs_rsi_macd_schaff = Input(shape=(None, 4))
#       inputs.append(inputs_rsi_macd_schaff)
#       input_bolinger = Input(shape=(None, 3))
#       inputs.append(input_bolinger)
#       input_adx = Input(shape(None, 3))
#       inputs.append(input_adx)
#       input_one_hot = Input(shape(None, 8))
#       inputs.append(input_one_hot)

#       x1 = Dense(6, activation='leaky relu')(input_price_info)
#       x2 = Dense(6, activation='leaky relu')(input_adx)
#       x3 = Dense(6, activation='leaky relu')(input_bolinger)
#       x4 = Dense(8, activation='leaky relu')(input_one_hot)

#       feature_vector = Concatenate(axis=1)(x1, inputs_rsi_macd_schaff, x2, x3, x4)
#       x = Dense(64, activation='relu')(feature_vector)
#       embedding = Dropout(0.5)(x)

#       layer_1 = LSTM(64, return_sequences=True)
#       x = layer_1(embedding, initial_state=encoder_states)
#       output_layer_1 = Activation('leaky relu')(x)

#       layer_2 = LSTM(32, return_sequences=True)
#       x = layer_2(output_layer_1)
#       output_layer_2 = Activation('leaky relu')(x)

#       final_dense = Dense(3, activation='linear')
#       final_output = final_dense(output_layer_2)

#       model = Model(inputs, final_output)
#       return model

# Take past 30 timesteps one hot veectors then use cnn and give it to the neural net 


class cnn_model(AbstractModelBuilder):

  def buildModel(self):
    hourly_data = Input(shape=(1, 30, 3))

    x = Conv2D(2, (1,3), activation='relu')(hourly_data)
    x = MaxPooling2D(pool_size=(1,2))(x)
    x = Conv2D(4, (1,3), activation='relu')(x)
    x = MaxPooling2D(pool_size=(1,2))(x)
    x = Conv2D(4, (1,3), activation='relu')(x)
    x = Flatten()(x)
    x0 = Dense(8, activation='relu')(x)

    input_PnL = Input(shape=(9,))
    x1 = Dense(4)(input_PnL)
    x1 = LeakyReLU()(x1)

    # scaling
    input_scale = Input(shape=(30,))
    x = Dense(4, activation='relu')(input_scale)
    x_scale = Dense(1)(x)

    inputs = [hourly_data, input_PnL, input_scale]
    input_price_info = Input(shape=(1, 60, 2))
    inputs.append(input_price_info)
    input_indicators = Input(shape=(1, 60, 5))
    inputs.append(input_indicators)
    input_bolinger = Input(shape=(4, 60, 1))
    inputs.append(input_bolinger)
    input_adx = Input(shape=(1, 60, 3))
    inputs.append(input_adx)
    input_nb = Input(shape=(1, 30, 2))
    inputs.append(input_nb)
    input_onehot = Input(shape=(10, 30, 1))
    inputs.append(input_onehot)

    # price
    x = Conv2D(5, (1,3), activation='relu')(input_price_info)
    x = Conv2D(8, (1,3), activation='relu')(x)
    x = MaxPooling2D(pool_size=(1,2), strides=(1,2))(x)
    x = Conv2D(12, (1,3), activation='relu')(x)
    x = Conv2D(14, (1,3), activation='relu')(x)
    x = MaxPooling2D(pool_size=(1,2), strides=(1,2))(x)
    x = Conv2D(16, (1,3), activation='relu')(x)
    x = MaxPooling2D(pool_size=(1,2), strides=(1,2))(x)
    x = Flatten()(x)
    x = Dense(25, activation='relu')(x)
    x_price = Dropout(0.3)(x)

    # nifty bank
    x = Conv2D(3, (1,3), activation='relu')(input_nb)
    x = MaxPooling2D(pool_size=(1,2), strides=(1,2))(x)
    x = Conv2D(4, (1,3), activation='relu')(x)
    x = MaxPooling2D(pool_size=(1,2), strides=(1,2))(x)
    x = Conv2D(5, (1,3), activation='relu')(x)
    x = Flatten()(x)
    x_nb = Dense(6)(x)

    # adx
    x = Dense(1)(input_adx)
    x = LeakyReLU()(x)
    indicators = Concatenate(axis=3)([input_indicators, x, input_adx[:,:,:,1:]])

    # bolinger
    x = Conv2D(3, (4,3), activation='relu')(input_bolinger)
    x = Conv2D(4, (1,5), activation='relu')(x)
    x = MaxPooling2D(pool_size=(1,2), strides=(1,2))(x)
    x = Conv2D(5, (1,7), activation='relu')(x)
    x = Conv2D(6, (1,10), activation='relu')(x)
    x = MaxPooling2D(pool_size=(1,2), strides=(1,2))(x)
    x = Flatten()(x)
    x = Dense(10, activation='relu')(x)
    x_bolinger = Dropout(0.3)(x)

    # onehot
    x = Conv2D(8, (8,3), activation='relu')(input_onehot)
    x = MaxPooling2D(pool_size=(1,2), strides=(1,2))(x)
    x = Conv2D(10, (1,3), activation='relu')(x)
    x = MaxPooling2D(pool_size=(1,2), strides=(1,2))(x)
    x = Conv2D(12, (1,5), activation='relu')(x)
    x = Flatten()(x)
    x = Dense(10, 'relu')(x)
    x_onehot = Dropout(0.2)(x)

    #indicators
    x = Conv2D(8, (1,3), activation='relu')(indicators)
    x = Conv2D(12, (1,3), activation='relu')(x)
    x = MaxPooling2D(pool_size=(1,2), strides=(1,2))(x) 
    x = Conv2D(12, (1,3), activation='relu')(x)
    x = Conv2D(16, (1,3), activation='relu')(x)
    x = MaxPooling2D(pool_size=(1,2), strides=(1,2))(x)
    x = Conv2D(20, (1,3), activation='relu')(x)
    x = MaxPooling2D(pool_size=(1,2), strides=(1,2))(x)
    x = Flatten()(x)
    x = Dense(20, activation='relu')(x)
    x_indicators = Dropout(0.3)(x)

    # concatenate and combine all the feature_vectors
    x = Concatenate(axis=1)([x_price, x_bolinger, x_indicators, x_onehot, x_nb])
    x = Dense(30)(x)
    x = LeakyReLU()(x)
    x = Dropout(0.5)(x)

    # concatenate context
    x = Concatenate(axis=1)([x, x1, x0])
    x = Dense(23)(x)
    x = LeakyReLU()(x)
    x = Dropout(0.5)(x)

    x = Dense(12)(x)
    x = LeakyReLU()(x)

    # Experiment with tanh too
    x = Dense(3, activation='linear')(x)

    # provide scaling
    # possible to force x_scale > 0 ? 
    final_output = x * x_scale

    # num trainable params large ~ 20k ? 
    model = Model(inputs, final_output)

    return model


  def compile_model(self):
    self.model.compile(loss='mse', optimizer='adam')

  def predict(self, data):   
    return self.model.predict([data['hourly_data'],   
                   data['portfolio'],
                   data['scale'], 
                   data['price_curve'],
                   data['indicators'],
                   data['bolinger'],
                   data['adx'],
                   data['nb'],
                   data['onehot']])


  def trainModel(self, data, targets):
    return self.model.train_on_batch([data['hourly_data'],    
                      data['portfolio'],
                      data['scale'], 
                      data['price_curve'],
                      data['indicators'],
                      data['bolinger'],
                      data['adx'],
                      data['nb'],
                      data['onehot']],
                      targets)
  
  def printSummary(self):
    print(self.model.summary())

In [61]:
# Improvements
# prioritized Experience Replay
# Dual Target Network
# Better Optimzation techique
class ExperienceReplay(object):
  
  def __init__(self, max_memory=100000, discount=.9):
    self.max_memory = max_memory
    self.count_range = {}
    self.memory = list()
    self.discount = discount
    self.record_sort = False

  def remember(self, states, game_over):
    # memory[i] = [[state_t, action_t, reward_t, state_t+1], game_over?]
    self.memory.append([states, game_over, 0])
    if self.record_sort:
      if self.count_range.__contains__(0):
        l, r = self.count_range[0]
        self.count_range[0] = (l, r + 1)
      else:
        self.count_range[0] = (len(self.memory) - 1, len(self.memory) - 1)
    
    if len(self.memory) > self.max_memory:
      del self.memory[:500]

  # keep record sorted
  def _adjust_record(self, idx):
    if not self.record_sort:
      return

    old_count = self.memory[idx][2]
    self.memory[idx][2] = old_count + 1
    old_l,old_r = self.count_range[old_count]
  
    if idx != old_l:
      temp = self.memory[idx]
      self.memory[idx] = self.memory[old_l]
      self.memory[old_l] = temp

    if self.count_range.__contains__(old_count + 1):
      new_l,new_r = self.count_range[old_count + 1]
      self.count_range[old_count + 1] = (new_l, new_r + 1)
    else:
      self.count_range[old_count + 1] = (old_l, old_l)

    if old_l + 1 > old_r:
      del self.count_range[old_count - 1]
    else:
      self.count_range[old_count - 1] = (old_l + 1, old_r)

  def get_batch(self, env, model_interface, batch_size=64):
    len_memory = len(self.memory)
    num_actions = model_interface.model.output_shape[-1]
    inputs = {}
    price_curve, indicators, bolinger, adx, nb, onehot, portfolio, hourly_data, scale_data = [], [], [], [], [], [], [], [], []

    targets = np.zeros((min(len_memory, batch_size), num_actions))
    for i, idx in enumerate(np.random.randint(0, len_memory, size=min(len_memory, batch_size))):
      pnl_state_t, action_t, reward_t, pnl_state_tp1, code, day, index = self.memory[idx][0]
      self._adjust_record(idx)
      state_t = env.generate_data(code, day, index)
      state_t['portfolio'] = pnl_state_t
      game_over = self.memory[idx][1]
      targets[i] = model_interface.predict(state_t)[0]
      if not game_over:
        state_tp1 = env.generate_data(code, day, index + 1)
        state_tp1['portfolio'] = pnl_state_tp1
        Q_sa = np.max(model_interface.predict(state_tp1)[0])
        targets[i, action_t] = reward_t + self.discount * Q_sa
      else:
        targets[i, action_t] = reward_t

      price_curve.append(np.squeeze(state_t['price_curve'], axis=0))
      indicators.append(np.squeeze(state_t['indicators'], axis=0))
      bolinger.append(np.squeeze(state_t['bolinger'], axis=0))
      adx.append(np.squeeze(state_t['adx'], axis=0))
      onehot.append(np.squeeze(state_t['onehot'], axis=0))
      nb.append(np.squeeze(state_t['nb'], axis=0))
      portfolio.append(np.squeeze(state_t['portfolio'], axis=0))
      hourly_data.append(np.squeeze(state_t['hourly_data'], axis=0))
      scale_data.append(np.squeeze(state_t['scale'], axis=0))


    inputs['price_curve'] = np.array(price_curve)
    inputs['indicators'] = np.array(indicators)
    inputs['bolinger'] = np.array(bolinger)
    inputs['adx'] = np.array(adx)
    inputs['onehot'] = np.array(onehot)
    inputs['portfolio'] = np.array(portfolio)
    inputs['nb'] = np.array(nb)
    inputs['scale'] = np.array(scale_data)
    inputs['hourly_data'] = np.array(hourly_data)

    return inputs, targets


In [64]:
path_list = ['BAJFINANCE/processed_data', 'AXISBANK/processed_data', 'HDFC/processed_data', 'HDFCBANK/processed_data', 'ICICIBANK/processed_data']
X_price_info, X_indicators, X_adx, X_bolinger, X_onehot, X_nb = [{} for _ in range(6)]
X_avg_val, X_high_val, X_low_val, X_close_val = [{} for _ in range(4)]

code = 0
for path in path_list:
  if not os.path.exists(path):
    print('path not found', path)
  else:
    f = open(path, 'rb')
    obj = pickle.load(f)
    X_price_info[code] = obj['X_price_info']
    X_indicators[code] = obj['X_indicators']
    X_adx[code] = obj['X_adx']
    X_bolinger[code] = obj['X_bolinger']
    X_onehot[code] = obj['X_onehot']
    X_nb[code] = obj['X_nb']
    X_avg_val[code] = obj['avg_val']
    X_high_val[code] = obj['high_val']
    X_low_val[code] = obj['low_val']
    X_close_val[code] = obj['close_val']
    code += 1
    f.close()

In [63]:
# parameters
epsilon = .5  # exploration
min_epsilon = 0.1
epoch = 2
max_memory = 5000
batch_size = 64
scope = 60
discount = 0.9    # increasing future discount incentivise future P

In [72]:
env = MarketEnv(X_price_info,
        X_indicators,
        X_bolinger,
        X_adx,
        X_onehot,
        X_nb,
        X_avg_val,
        X_close_val,
        X_high_val,
        X_low_val,
        scope,
        sudden_death = -1.0)

In [67]:
# from keras.optimizers import SGD
modelFilename='cnn_model.h5'
model_interface = cnn_model(modelFilename)
model_interface.loadModel()
model_interface.compile_model()

# sgd = SGD(lr = 0.001, decay = 1e-6, momentum = 0.9, nesterov = True)
# need a better optimization/random optimization techique.
# maybe try Particle swarm or genectic algorithm

In [68]:
# Initialize experience replay object
exp_replay = ExperienceReplay()
# Train
win_cnt = 0
train_cycle = 30

In [69]:
exp_replay

In [70]:
env

In [76]:
model_interface.predict(input_t)

array([[ 1.4604977e-04,  2.3279474e-05, -7.8652592e-05]], dtype=float32)

In [77]:
model_interface.predict(input_tm1)

array([[ 1.7858001e-04,  3.6566528e-06, -9.8831340e-05]], dtype=float32)

In [78]:
model_interface.model.save_weights("model_checkpoint_day_155.h5" if modelFilename == None else modelFilename, overwrite=True)

In [73]:
epoch = 1
for e in range(epoch):
  loss = 0.
  day = 0
  game_over = False
  delay = True
  env.reset()

  while not game_over:
    
    input_t = env.nextDay()
    day_end = False
    counter = 0
    cumReward = 0
    realized_pnl = 0
    if delay and len(exp_replay.memory) > 30000:
      print('switching training delay off..')
      delay = False

    while not day_end:

      input_tm1 = input_t
      isRandom = False
      q = [None]

      # get next action
      if np.random.rand() <= epsilon or delay:
        action = np.random.randint(0, len(env.actions), size=1)[0]
        isRandom = True
      else:
        q = model_interface.predict(input_tm1)
        action = np.argmax(q[0])

      if np.nan in q:
        print("OCCUR NaN!!!", str(counter), str(day), str(input_tm1['code']))
        # exit()

      # apply action, get rewards and new state
      input_t, reward, day_end, game_over, info, pnl = env.step(action)
      cumReward += reward
      realized_pnl += pnl

      # store experience
      exp_replay.remember([input_tm1['portfolio'],
                action,
                reward,
                input_t['portfolio'],
                input_tm1['code'],
                input_tm1['day'],
                input_tm1['index']],
                day_end)

      # adapt model
      if counter % train_cycle == 0 and not delay:
        inputs, targets = exp_replay.get_batch(env, model_interface)
        loss = model_interface.trainModel(inputs, targets)
        print('batch = ', str(counter // train_cycle), 'loss = ', str(loss))
      
      counter += 1

    day += 1
    print('day = ', str(day), 'cumReward =', str(cumReward), 'pnl =', str(realized_pnl))

    if cumReward > 0:
      win_cnt += 1

  print("Epoch {:03d}/{} | Loss {:.4f} | Win count {} | Epsilon {:.4f}".format(e, epoch, loss, win_cnt, epsilon))
  # Save trained model weights and architecture, this will be used by the visualization code
  model_interface.model.save_weights("model.h5" if modelFilename == None else modelFilename, overwrite=True)
  epsilon = max(min_epsilon, epsilon * 0.99)

day =  1 cumReward = 0.013450827067096383 pnl = 0.004149384049375149
day =  2 cumReward = -0.02846824665938035 pnl = -0.03248094130769108
day =  3 cumReward = 0.013805230808175552 pnl = 0.008900586010251121
day =  4 cumReward = 0.08823392060068269 pnl = 0.05799911419642656
day =  5 cumReward = -0.008236795590402482 pnl = -0.012385912192470205
day =  6 cumReward = 0.01805715597449166 pnl = 0.0024780707601942398
day =  7 cumReward = 0.054442825751373936 pnl = 0.03271713296647727
day =  8 cumReward = 0.0068140603037950675 pnl = -0.0031255762698507716
day =  9 cumReward = 0.00881053589239995 pnl = 0.0020726299111311075
day =  10 cumReward = 0.03728889865915179 pnl = 0.018338434667893866
day =  11 cumReward = 0.03400846465400684 pnl = 0.019372358253247235
day =  12 cumReward = 0.04391469264112428 pnl = 0.025435898318079797
day =  13 cumReward = -0.012538540081231538 pnl = -0.019479459872003724
day =  14 cumReward = -0.013837073573874325 pnl = -0.01864241897127028
day =  15 cumReward = 0.030

KeyboardInterrupt: ignored

In [71]:
# import gym
# from gym import spaces

# idea don't penalize strictly on holding even when the price falls upto a certain level
# use the idea of stoploss i.e when the price fall below stoploss start penalizing heavily for subsequent drops
# create levels:

# rise :
# All values  :  MULTIPLIER = 0.9

# fall :
# [   > 1.5 )%  :  MULTIPLIER = 1.5
# [1.5 - 0.5)%  :  MULTIPLIER = 1.0
# [0.5 - 0)%    :  MULTIPLIER = 0.8
# [0 -  0.5)%   :  MULTIPLIER = 0.2
# [0.5  - 1)%   :  MULTIPLIER = 1.0
# [1 -  1.5)%   :  MULTIPLIER = 2.0
# [1.5 -  2)%   :  MULTIPLIER = 3.0
# [ >=   2 )%   :  MULTIPLIER = 4.0
# [ may force it to sell ] ?

class MarketEnv(object):

    PENALTY = 1 #0.999756079
    ON_PROFIT_HOLDING = 0.9

    def __init__(self, X_price_info, X_indicators, X_bolinger, X_adx, X_onehot, X_nb, X_avg_val, X_close_val, X_high_val, X_low_val, scope, sudden_death = -1., cumulative_reward = False):
        self.X_price_info = X_price_info
        self.X_indicators = X_indicators
        self.X_bolinger = X_bolinger
        self.X_adx = X_adx
        self.X_onehot = X_onehot
        self.X_nb = X_nb
        self.X_avg_val = X_avg_val
        self.X_close_val = X_close_val
        self.X_high_val = X_high_val
        self.X_low_val = X_low_val
        self.sudden_death = sudden_death
        self.cumulative_reward = cumulative_reward
        self.scope = scope
        self.num_target = X_price_info.__len__()

        self.actions = [
            "LONG",
            "HOLD",
            "SHORT"
        ]

        self.ON_LOSS_HOLDING = [1.5, 1.2, 1.0, 0.8, 0.25, 0.9, 2.0, 3.0, 4.0]

        # self.action_space = spaces.Discrete(len(self.actions))
        # self.observation_space = spaces.Box(np.ones(scope * (len(input_codes) + 1)) * -1, np.ones(scope * (len(input_codes) + 1)))

        # self.reset()
        self._seed()

    def _get_holding_mutiplier(self, cum_reward, diff):     
        if diff >= 0:
            return self.ON_PROFIT_HOLDING

        return self.ON_LOSS_HOLDING[min(max(int((2.0 - cum_reward) / 0.5), 0), 8)]

    def _create_hourly_price_till(self, index, timesteps, code):
        minutes = timesteps * 60
        end = min(len(self.X_close_val[code]), index + 1)
        start = max((index + 1) % 60, index - minutes + 1)
        j = 0
        hourly_avg_price, hourly_close_price, hourly_high_price, hourly_low_price = [np.zeros(timesteps) for _ in range(4)]

        for i in np.arange(start, end, 60):
            hourly_avg_price[j] = np.average(self.X_avg_val[code][i: i + 60])
            hourly_close_price[j] = self.X_close_val[code][i + 59]
            hourly_high_price[j] = np.max(self.X_high_val[code][i: i + 60])
            hourly_low_price[j] = np.min(self.X_low_val[code][i: i + 60])
            j += 1
        
        return (hourly_avg_price, hourly_close_price, hourly_high_price, hourly_low_price, j)

    def step(self, action):
        if self.done:
            return self.state, self.reward, False, self.done, {}

        vari = self.X_price_info[self.targetCode][self.currentDay][self.currentTargetIndex][1] / 100
        self.reward = 0.
        list_length = len(self.boughts)
        self.budget = self.budget / (1 + vari)  # iter 0?
        temp = 0.

        for i in range(list_length):
            old_pl = self.boughts[i]
            self.boughts[i] = self.boughts[i] * MarketEnv.PENALTY * (1 + vari * self.position)
            temp += self.boughts[i] - old_pl

        # pnl of currentposition
        cum_reward = sum(self.boughts) * self.position - len(self.boughts)
        realized_pnl = 0.

        if self.actions[action] == "LONG":
            if self.position == -1:
                for b in self.boughts:
                    self.budget += -b
                    self.reward += -(b + 1)
                
                realized_pnl = self.reward
                if self.cumulative_reward:
                    self.reward = self.reward / max(1, len(self.boughts))
                
                # if got bankrupt
                if self.sudden_death * len(self.boughts) > self.reward:
                    self.nextDay = True

                self.boughts = []
                self.position = 0
            elif self.budget > 0:
                self.boughts.append(1.0)
                self.position = 1
                self.budget -= 1.
        
        elif self.actions[action] == "SHORT":
            if self.position == 1:
                for b in self.boughts:
                    self.budget += b
                    self.reward += b - 1
                
                realized_pnl = self.reward
                if self.cumulative_reward:
                    self.reward = self.reward / max(1, len(self.boughts))

                if self.sudden_death * len(self.boughts) > self.reward:
                    self.nextDay = True

                self.boughts = []
                self.position = 0
            elif self.budget > 0:
                self.boughts.append(-1.0)
                self.position = -1
                self.budget -= 1.
        
        else:
            temp = temp * self.position
            self.reward = self._get_holding_mutiplier(cum_reward, temp) * temp

        self.currentTargetIndex += 1

        if self.currentTargetIndex >= self.episode_length or (self.budget < 1. and self.position == 0):
            self.day_index += 1
            self.nextday = True

        if self.day_index >= self.num_days:
            self.done = True
        
        self._defineState()
        return self.state, self.reward, self.nextday, self.done, self.targetCode, realized_pnl

        # ignoring this for the moment
        # if self.done:
            # for b in self.boughts:
            #   self.reward += (b * self.position) - 1
            # if self.cumulative_reward:
            #   self.reward = self.reward / max(1, len(self.boughts))

            # self.boughts = []

    def reset(self):
        self.targetCode = random.randint(0, self.num_target - 1)
        self.num_days = self.X_price_info[self.targetCode].shape[0]
        self.episode_length = self.X_price_info[self.targetCode].shape[1]
        # ignore day {0, 1, 2}
        self.attack_sequence = np.random.permutation([i for i in range(3, self.num_days)])
        self.currentTargetIndex = 0
        self.currentDay = 0
        self.day_index = 0
        self.position = 0
        self.boughts = []
        self.nextday = False
        self.done = False
        self.reward = 0.

    def _render(self, mode='human', close=False):
        if close:
            return
        return self.state

    def _seed(self):
        return int(random.random() * 100)

    def nextDay(self):
        self.currentTargetIndex = self.scope - 1
        self.currentDay = self.attack_sequence[self.day_index]
        self.nextday = False
        self.boughts = []
        self.reward = 0.
        self.position = 0
        self.budget = 8.        # automation required # meaning power to buy x shares at current price
        self._defineState()

        return self.state

    def _create_hourly_data(self, code, day, index):
        timesteps = 80
        lookback = self.scope // 2
        # adjusting day_ignored and index_ignored
        end = (day + 4) * 375 + index + 30
        hourly_avg_val, hourly_close_val, hourly_high_val, hourly_low_val, j = self._create_hourly_price_till(end, timesteps, code)
        hourly_avg_val = hourly_avg_val[:j]
        # if j != timesteps:
          # print('lookback = ', str(lookback), 'length=', str(j), 'timestep=', str(timesteps), 'currentIndex =', str(index + 30), 'currentDay =', str(day + 4))
        hourly_rsi = (RSI(hourly_close_val[:j], timeperiod=14)[-lookback:]) / 100
        hourly_atr = NATR(hourly_high_val[:j], hourly_low_val[:j], hourly_close_val[:j], timeperiod=14)[-lookback:]
        X_avg_price = (hourly_avg_val[-lookback:] - hourly_avg_val[-(lookback + 1) : -1]) * 10 / hourly_avg_val[-(lookback + 1) : -1]

        X_avg_price = np.expand_dims(X_avg_price, axis=(0, 2))
        hourly_rsi = np.expand_dims(hourly_rsi, axis=(0, 2))
        hourly_atr = np.expand_dims(hourly_atr, axis=(0, 2))
        hourly_data = np.expand_dims(np.concatenate([X_avg_price, hourly_rsi, hourly_atr], axis=2), axis=0)

        return hourly_data

    def _standardize(self, X_array):
        n = len(X_array[0])
        for i in range(n):
            temp = X_array[:, i]
            X_array[:, i] = (temp - np.mean(temp)) / np.std(temp)

    # to prevent memory overflow create training data in an online way
    def generate_data(self, code, day, index):
        r_data = {}

        if index < self.scope - 1:
            print('index not in range =', str(index))
            return None

        # normalize prices
        price_curve = np.array(X_price_info[code][day][index - self.scope + 1 : index + 1])
        scaling = np.array(price_curve[self.scope // 2:, 1])
        r_data['scale'] = np.expand_dims(scaling, axis=0)
        self._standardize(price_curve)
        price_curve = np.expand_dims(price_curve, axis=(0, 1))

        # normalize nb prices
        nb_price = np.array(X_price_info[code][day][index - (self.scope // 2) + 1 : index + 1])
        self._standardize(nb_price)
        nb_price = np.expand_dims(nb_price, axis=(0, 1))

        indicators = X_indicators[code][day][index - self.scope + 1 : index + 1]
        indicators = np.expand_dims(indicators, axis=(0, 1))
        
        bolinger = X_bolinger[code][day][index - self.scope + 1 : index + 1]
        bolinger = np.swapaxes(bolinger, 0, 1)
        bolinger = np.expand_dims(bolinger, axis=(0, 3))
        
        adx = X_adx[code][day][index - self.scope + 1 : index + 1]
        adx = np.expand_dims(adx, axis=(0, 1))
        
        onehot = X_onehot[code][day][index - (self.scope // 2) + 1 : index + 1]
        onehot = np.swapaxes(onehot, 0, 1)
        onehot = np.expand_dims(onehot, axis=(0, 3))

        r_data['price_curve'] = price_curve
        r_data['indicators'] = indicators
        r_data['bolinger'] = bolinger
        r_data['adx'] = adx
        r_data['onehot'] = onehot
        r_data['nb'] = nb_price
        r_data['hourly_data'] = self._create_hourly_data(code, day, index)

        return r_data

    def _defineState(self):
        if self.nextday or self.done:
            return

        this_PnL = (sum(self.boughts) * self.position - len(self.boughts)) / 8

        tmpState = self.generate_data(self.targetCode, self.currentDay, self.currentTargetIndex)
        tmpState['code'] = self.targetCode
        tmpState['day'] = self.currentDay
        tmpState['index'] = self.currentTargetIndex
        # markovian in nature
        onehot_code = np.zeros(self.num_target, dtype=float)
        onehot_code[self.targetCode] = 1.0
        tmpState['portfolio'] = np.expand_dims(np.concatenate([np.array([this_PnL,
                                                            self.budget / 8,
                                                            self.position,
                                                            float(self.currentDay) / self.episode_length]),
                                                            onehot_code], axis=0), axis=0)
        self.state = tmpState